# Лабораторная работа 4
## Генералов Даниил, 1032212280

> В данной лабораторной работе необходимо для какого-либо метода из лабораторной №3 составить JSON схему, описывающую все возможности метода. JSON схема должна быть составлена и для объекта, который посылается в запросе и для объекта, который приходит в ответе от сервера. Правильность схемы проверьте с помощью валидатора, например jsonschemavalidator.net
> 
> Официальный сайт json-schema.org. Перевод документации см. в PDF файле, приложенном к лабораторной работе.
> 
> Те методы, для которых есть JSON схема в явном виде на странице документации ссылка выбирать нельзя.


> Исходный код этого файла доступен в git-репозитории: https://github.com/danya02/rudn-year3-api-labs/blob/main/lab4/main.ipynb

В этой лабораторной работе следует сделать JSON-Schema для описания какого-то из методов, которые мы использовали в предыдущей лабораторной работе.
Я решил описать метод `docs.get`, потому что он имеет сравнительно мало элементов в своей структуре, но при этом эти элементы ссылаются друг на друга, что позволит продемонстрировать различные способности JSON-Schema.

Сначала нужно подключить библиотеку для валидации документов по JSON-Schema:

In [1]:
!pip3 install jsonschema==4.21.1
!pip3 install referencing==0.34.0

import jsonschema

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Сначала нужно определить тип запроса. Запрос обычно передается как параметры query-строки, но также может быть передан в теле запроса как WWW-form.
Если представить, что API также принимал бы JSON-документы, то спецификация запроса выглядела бы так:

In [2]:
request_schema = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/docs.get/request",
    "type": "object",
    "properties": {
        "v": {
            "type": "string",
            "description": "Версия используемого API",
            "const": "5.199"
        },
        "access_token": {
            "type": "string",
            "description": "Токен доступа (должен быть пользовательский)",
        },
        "count": {
            "type": "integer",
            "description": "Сколько документов вернуть?",
            "minimum": 1,
            "maximum": 2000
        },
        "offset": {
            "type": "integer",
            "description": "С какого документа по порядку вернуть?",
            "minimum": 0,
            "maximum": 1999
        },
        "type": {
            "type": "integer",
            "description": "Тип документов, которые следует возвращать",
            "enum": [1,2,3,4,5,6,7]
        },
        "owner_id": {
            "type": "integer",
            "description": "Документы, принадлежащие пользователю с ID или группе с -ID (если нет, то для пользователя-владельца токена)",
        },
        "return_tags": {
            "type": "boolean",
            "description": "??? незадокументированно ???"
        }
    },
    "required": ["v", "access_token"]
}

Теперь можно проверить некоторые документы с помощью этой схемы:

In [3]:
jsonschema.validate({"v": "5.199", "access_token": "abc123"}, request_schema)

In [4]:
import traceback
try:
    jsonschema.validate({"owner_id": 123}, request_schema)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_2394150/2102746917.py", line 3, in <module>
    jsonschema.validate({"owner_id": 123}, request_schema)
  File "/home/danya/.local/lib/python3.11/site-packages/jsonschema/validators.py", line 1312, in validate
    raise error
jsonschema.exceptions.ValidationError: 'v' is a required property

Failed validating 'required' in schema:
    {'$id': 'https://vk.com/api/docs.get/request',
     '$schema': 'https://json-schema.org/draft/2020-12/schema',
     'properties': {'access_token': {'description': 'Токен доступа (должен '
                                                    'быть '
                                                    'пользовательский)',
                                     'type': 'string'},
                    'count': {'description': 'Сколько документов вернуть?',
                              'maximum': 2000,
                              'minimum': 1,
                              'type': 'integer'},
            

In [5]:
try:
    jsonschema.validate({"v": "5.199", "access_token": "abc123", "owner_id": 123, "type": 12}, request_schema)
except:
    traceback.print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_2394150/543512951.py", line 2, in <module>
    jsonschema.validate({"v": "5.199", "access_token": "abc123", "owner_id": 123, "type": 12}, request_schema)
  File "/home/danya/.local/lib/python3.11/site-packages/jsonschema/validators.py", line 1312, in validate
    raise error
jsonschema.exceptions.ValidationError: 12 is not one of [1, 2, 3, 4, 5, 6, 7]

Failed validating 'enum' in schema['properties']['type']:
    {'description': 'Тип документов, которые следует возвращать',
     'enum': [1, 2, 3, 4, 5, 6, 7],
     'type': 'integer'}

On instance['type']:
    12


После этого следует описать результат. Он может быть либо ошибкой, либо успешным; сначала рассмотрим ситуацию ошибок.

In [6]:
error_schema = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/generic.error",
    "type": "object",
    "properties": {
        "error_code": {
          "type": "integer",
          "description": "Код ошибки",
        },
        "error_msg": {
          "type": "string",
          "description": "Человекочитаемое сообщение об ошибке"
        },
        "request_params": {
          "type": "array",
          "description": "Копия параметров, которые были отправлены в запросе",
          "items": {
            "type": "object",
            "properties": {
                "key": {"type": "string"},
                "value": {"type": "string"},
            },
            "additionalProperties": False,
            "required": ["key", "value"],
          },
        }
    },
    "required": ["error_code", "error_msg"],
    "additionalProperties": True,
}

Поскольку наши схемы будут становиться связанными между собой, следует теперь добавить механизм для связывания схем вместе.
Используя эту библиотеку, нужно создать `Registry`, которая содержит нужные ресурсы.

In [7]:
import jsonschema.exceptions
import referencing
import referencing.jsonschema

GLOBAL_RESOURCES = {}

def add_global_resource(schema):
    GLOBAL_RESOURCES[schema['$id']] = referencing.jsonschema.DRAFT202012.create_resource(schema)

def assert_ok(schema, doc):
    reg = referencing.Registry().with_resources(list(GLOBAL_RESOURCES.items()))
    validator = jsonschema.Draft202012Validator(schema, registry=reg)
    validator.validate(doc)

def assert_ng(schema, doc):
    try:
        assert_ok(schema, doc)
    except jsonschema.exceptions.ValidationError:
        return
    else:
        raise ValueError("Должно было быть неверно")

In [8]:
add_global_resource(error_schema)

assert_ok(error_schema, {"error_code": 123, "error_msg": "Что-то не так"})
assert_ok(error_schema, {"error_code": 123, "error_msg": "Что-то не так", "request_params": []})

assert_ng(error_schema, {"error_code": 123, "error_msg": "Что-то не так", "request_params": [{}]})

assert_ok(error_schema, {"error_code": 123, "error_msg": "Что-то не так", "request_params": [{"key": "abc", "value": "123"}]})
assert_ok(error_schema, {"error_code": 123, "error_msg": "Что-то не так", "request_params": [{"key": "abc", "value": "123"}, {"key": "def", "value": "456"}]})

assert_ng(error_schema, {"error_code": 123, "error_msg": "Что-то не так", "request_params": [{"key": "abc", "value": "123"}, {"key": "def", "value": "456", "extra": "wtf"}]})

assert_ok(error_schema, {"error_code": 123, "error_msg": "Что-то не так", "request_params": [], "extra": "OK"})

print("It didn't crash!")

It didn't crash!


При успешном выполнении возвращается массив объектов-"файлов", поэтому дальше мы будем описывать их.
Они содержат некоторую информацию, в том числе, если это фотография -- массив копий изображения.
Поэтому мы опишем их дальше.

In [9]:
photo_thumbnail_schema = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/generic.photo_sizes",
    "type": "object",
    "properties": {
        "src": {
            "type": "string",
            "description": "Путь к этой копии изображения",
            # регулярное выражение из http://stackoverflow.com/questions/3809401
            "pattern": r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
        },
        "width": {
            "type": "integer",
            "description": "Ширина копии в пикселях (или ноль, если неизвестно для фоток раньше 2012)",
            "minimum": 0,
        },
        "height": {
            "type": "integer",
            "description": "Высота копии в пикселях (или ноль, если неизвестно для фоток раньше 2012)",
            "minimum": 0,
        },
        "type": {
            "type": "string",
            "description": "Тип этой копии, описывает пропорциональное свойство этой копии",
            "enum": ["s","m","x","o","p","q","r","y","z","w"]
        }
    },
    "required": ["src", "width", "height", "type"]
}
add_global_resource(photo_thumbnail_schema)

In [10]:
assert_ok(photo_thumbnail_schema, {"src": "https://example.com/1.jpg", "width": 1920, "height": 1080, "type": "w"})

assert_ng(photo_thumbnail_schema, {"src": "этот URL поддельный", "width": 1920, "height": 1080, "type": "w"})
assert_ng(photo_thumbnail_schema, {"src": "ftp://example.com/1.jpg", "width": 1920, "height": 1080, "type": "w"})
assert_ng(photo_thumbnail_schema, {"src": "https://example.com/1.jpg", "width": 1920, "height": 1080, "type": "?"})


assert_ok(photo_thumbnail_schema, {"src": "https://example.com/1.jpg", "width": 0, "height": 0, "type": "w"})

assert_ng(photo_thumbnail_schema, {"src": "https://example.com/1.jpg", "width": -1, "height": -1, "type": "w"})

print("It didn't crash!")

It didn't crash!


Теперь мы можем описать тип документа файла:

In [11]:
file_schema = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/generic.file",
    "type": "object",
    "properties": {
        "id": {
            "type": "integer",
            "description": "Идентификатор файла"
        },
        "owner_id": {
            "type": "integer",
            "description": "Идентификатор пользователя, загрузившего файл"
        },
        "title": {
            "type": "string",
            "description": "Название файла"
        },
        "size": {
            "type": "integer",
            "description": "Размер файла в байтах",
            "minimum": 0
        },
        "ext": {
            "type": "string",
            "description": "Расширение файла"
        },
        "date": {
            "type": "integer",
            "description": "Дата добавления в формате Unixtime",
            "minimum": 0
        },
        "url": {
            "type": "string",
            "description": "Адрес файла, по которому его можно загрузить",
            # регулярное выражение из http://stackoverflow.com/questions/3809401
            "pattern": r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
        },
        "type": {
            "type": "integer",
            "description": "Тип файла",
            "enum": [1,2,3,4,5,6,7,8],
        },
        "preview": {
            "description": "Информация для предварительного просмотра файла. Зависит от типа файла.",
            "anyOf": [  # один объект может иметь несколько таких
                {"$ref": "https://vk.com/api/generic.file/preview/photo"},
                {"$ref": "https://vk.com/api/generic.file/preview/video"},
                {"$ref": "https://vk.com/api/generic.file/preview/graffiti"},
                {"$ref": "https://vk.com/api/generic.file/preview/audio_message"},
            ]
        }
    },
    "required": ["id", "owner_id", "ext", "date", "size"]
}
add_global_resource(file_schema)

Также требуется добавить описание каждого из видов preview:

In [12]:
photo_preview = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/generic.file/preview/photo",
    "description": "Preview для фотографий",
    "properties": {
        "photo": {
            "type": "object",
            "required": ["sizes"],
            "properties": {
                "sizes": {
                    "type": "array",
                    "items": {
                        "$ref": "https://vk.com/api/generic.photo_sizes"
                    }
                }
            },
            "additionalProperties": False,
        }
    }, 
    "required": ["photo"],
    "additionalProperties": True,
}
add_global_resource(photo_preview)

In [13]:
video_preview = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/generic.file/preview/video",
    "description": "Preview для граффити",
    "properties": {
        "video": {
            "type": "object",
            "required": ["src", "width", "height"],
            "properties": {
                "src": {
                    "type": "string",
                    "description": "URL файла с видео",
                    # регулярное выражение из http://stackoverflow.com/questions/3809401
                    "pattern": r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
                },
                "width": {
                    "type": "integer",
                    "minimum": 1,
                    "description": "Ширина видео в px"
                },
                "height": {
                    "type": "integer",
                    "minimum": 1,
                    "description": "Высота видео в px"
                },
                "size": {
                    "type": "integer",
                    "description": "Размер видео в байтах",
                    "minimum": 0
                },
            },
            "additionalProperties": False,
        }
    }, 
    "required": ["video"],
    "additionalProperties": True,
}
add_global_resource(video_preview)

In [14]:
graffiti_preview = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/generic.file/preview/graffiti",
    "description": "Preview для граффити",
    "properties": {
        "graffiti": {
            "type": "object",
            "required": ["src", "width", "height"],
            "properties": {
                "src": {
                    "type": "string",
                    "description": "URL файла с граффити",
                    # регулярное выражение из http://stackoverflow.com/questions/3809401
                    "pattern": r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
                },
                "width": {
                    "type": "integer",
                    "minimum": 1,
                    "description": "Ширина изображения в px"
                },
                "height": {
                    "type": "integer",
                    "minimum": 1,
                    "description": "Высота изображения в px"
                }
            },
            "additionalProperties": False,
        }
    }, 
    "required": ["graffiti"],
    "additionalProperties": True,
}
add_global_resource(graffiti_preview)

In [15]:
audio_message_preview = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/generic.file/preview/audio_message",
    "description": "Preview для аудио-сообщения",
    "properties": {
        "audio_message": {
            "type": "object",
            "required": ["duration", "waveform", "link_ogg", "link_mp3"],
            "properties": {
                "link_ogg": {
                    "type": "string",
                    "description": "URL .ogg-файла",
                    # регулярное выражение из http://stackoverflow.com/questions/3809401
                    "pattern": r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
                },
                "link_mp3": {
                    "type": "string",
                    "description": "URL .mp3-файла",
                    # регулярное выражение из http://stackoverflow.com/questions/3809401
                    "pattern": r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)",
                },
                "duration": {
                    "type": "integer",
                    "minimum": 1,
                    "description": "Длительность аудиосообщения в секундах"
                },
                "waveform": {
                    "type": "array",
                    "description": "Массив значений для визуального отображения звука",
                    "items": {
                        "type": "integer"
                    }
                }
            },
            "additionalProperties": False,
        }
    }, 
    "required": ["audio_message"],
    "additionalProperties": True,
}
add_global_resource(audio_message_preview)

Теперь можно добавить общую форму ответа сервера -- либо успех, либо ошибку:

In [16]:
global_response_schema = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "$id": "https://vk.com/api/docs.get/response",
    "type": "object",
    "oneOf": [
        {
            "type": "object",
            "properties": {
                "error": {
                    "$ref": "https://vk.com/api/generic.error"
                }
            },
            "required": ["error"],
            "additionalProperties": False,
        },
        {
            "type": "object",
            "properties": {
                "response": {
                    "type": "object",
                    "additionalProperties": False,
                    "required": ["count", "items"],
                    "properties": {
                        "count": {"type": "integer", "minimum": 0},
                        "items": {"type": "array", "items": {"$ref": "https://vk.com/api/generic.file"}}
                    },
                }
            },
            "required": ["response"],
            "additionalProperties": False,
        }

    ]

}
add_global_resource(global_response_schema)

Наконец, можно проверять глобальный тип результата (а также зависимые типы) на нескольких примерах, чтобы убедиться, что он работает правильно:

In [17]:
assert_ok(global_response_schema,
{'response': {'count': 0, 'items': []}}
)

assert_ok(photo_preview, 
          {
              'photo': {'sizes': [{'height': 73,
                                    'src': 'https://sun9-32.userapi.com/c909618/u514286230/d15/-3/m_1900c3ab1c.jpg',
                                    'type': 'm',
                                    'width': 130},
                                    {'height': 56,
                                    'src': 'https://sun9-79.userapi.com/c909618/u514286230/d15/-3/s_1900c3ab1c.jpg',
                                    'type': 's',
                                    'width': 100},
                                    {'height': 123,
                                    'src': 'https://sun9-63.userapi.com/c909618/u514286230/d15/-3/o_1900c3ab1c.jpg',
                                    'type': 'o',
                                    'width': 220},
                                    {'height': 123,
                                    'src': 'https://sun9-25.userapi.com/c909618/u514286230/d15/-3/x_1900c3ab1c.jpg',
                                    'type': 'x',
                                    'width': 220}]}
          })

assert_ok(file_schema, 
                {'can_manage': True,
                        'date': 1633614805,
                        'ext': 'gif',
                        'id': 616801814,
                        'is_unsafe': 0,
                        'owner_id': 450410456,
                        'preview': {'photo': {'sizes': [{'height': 73,
                                                        'src': 'https://sun9-32.userapi.com/c909618/u514286230/d15/-3/m_1900c3ab1c.jpg',
                                                        'type': 'm',
                                                        'width': 130},
                                                        {'height': 56,
                                                        'src': 'https://sun9-79.userapi.com/c909618/u514286230/d15/-3/s_1900c3ab1c.jpg',
                                                        'type': 's',
                                                        'width': 100},
                                                        {'height': 123,
                                                        'src': 'https://sun9-63.userapi.com/c909618/u514286230/d15/-3/o_1900c3ab1c.jpg',
                                                        'type': 'o',
                                                        'width': 220},
                                                        {'height': 123,
                                                        'src': 'https://sun9-25.userapi.com/c909618/u514286230/d15/-3/x_1900c3ab1c.jpg',
                                                        'type': 'x',
                                                        'width': 220}]},
                                    'video': {'file_size': 44649,
                                            'height': 122,
                                            'src': 'https://vk.com/doc450410456_616801814?hash=RCt1QUMD5lAtGqm2QZYHtB9GACwGRvggGKekzr43Dzg&dl=GQ2TANBRGA2DKNQ:1713635885:Z9ty6RkypkCs1Oy7ZhSMH7DZZm3vctgkwLRGMbzxZMk&api=1&mp4=1',
                                            'width': 220}},
                        'size': 221982,
                    }
)


assert_ok(global_response_schema, {
    "response": {"count": 1, "items": [
        {
            "id": 123,
            "owner_id": 456,
            "ext": "graf",
            "date": 123456789,
            "size": 4096,
            "preview": {"graffiti": {
                        "src": "https://example.com/file.graf",
                        "width": 12,
                        "height": 34,
                    }}
        },
        {
            "id": 124,
            "owner_id": 456,
            "ext": "graf",
            "date": 123456780,
            "size": 8192,
            "preview": {"audio_message": {
                        "link_ogg": "https://example.com/msg.ogg",
                        "link_mp3": "https://example.com/msg.mp3",
                        "duration": 10,
                        "waveform": [1,2,3,4,5,6]
                    }}
        },

    ]}
})

print("It did not crash!")

It did not crash!


In [18]:
# получено из API
assert_ok(global_response_schema, 
{'error': {'error_code': 15,
		           'error_msg': 'Access denied',
		           'request_params': [{'key': 'v', 'value': '5.199'},
		                              {'key': 'count', 'value': '5'},
		                              {'key': 'owner_id', 'value': '1'},
		                              {'key': 'type', 'value': '3'},
		                              {'key': 'method', 'value': 'docs.get'},
		                              {'key': 'oauth', 'value': '1'}]}})

# получено из API
assert_ok(global_response_schema,
{'response': {'count': 3,
		              'items': [{'can_manage': True,
		                         'date': 1633614805,
		                         'ext': 'gif',
		                         'id': 616801814,
		                         'is_unsafe': 0,
		                         'owner_id': 450410456,
		                         'preview': {'photo': {'sizes': [{'height': 73,
		                                                          'src': 'https://sun9-32.userapi.com/c909618/u514286230/d15/-3/m_1900c3ab1c.jpg',
		                                                          'type': 'm',
		                                                          'width': 130},
		                                                         {'height': 56,
		                                                          'src': 'https://sun9-79.userapi.com/c909618/u514286230/d15/-3/s_1900c3ab1c.jpg',
		                                                          'type': 's',
		                                                          'width': 100},
		                                                         {'height': 123,
		                                                          'src': 'https://sun9-63.userapi.com/c909618/u514286230/d15/-3/o_1900c3ab1c.jpg',
		                                                          'type': 'o',
		                                                          'width': 220},
		                                                         {'height': 123,
		                                                          'src': 'https://sun9-25.userapi.com/c909618/u514286230/d15/-3/x_1900c3ab1c.jpg',
		                                                          'type': 'x',
		                                                          'width': 220}]},
		                                     'video': {'file_size': 44649,
		                                               'height': 122,
		                                               'src': 'https://vk.com/doc450410456_616801814?hash=RCt1QUMD5lAtGqm2QZYHtB9GACwGRvggGKekzr43Dzg&dl=GQ2TANBRGA2DKNQ:1713635885:Z9ty6RkypkCs1Oy7ZhSMH7DZZm3vctgkwLRGMbzxZMk&api=1&mp4=1',
		                                               'width': 220}},
		                         'size': 221982,
		                         'title': '1633614801030.gif',
		                         'type': 3,
		                         'url': 'https://vk.com/doc450410456_616801814?hash=RCt1QUMD5lAtGqm2QZYHtB9GACwGRvggGKekzr43Dzg&dl=DFtZUEyut6CF55UVNsLbcIgtgmCv0v3ZuFaCdzz4aWg&api=1&no_preview=1'},
		                        {'can_manage': True,
		                         'date': 1575491926,
		                         'ext': 'gif',
		                         'id': 528688101,
		                         'is_unsafe': 0,
		                         'owner_id': 450410456,
		                         'preview': {'photo': {'sizes': [{'height': 95,
		                                                          'src': 'https://sun9-65.userapi.com/c237331/u663156600/d59/-3/m_4327e2112b.jpg',
		                                                          'type': 'm',
		                                                          'width': 130},
		                                                         {'height': 73,
		                                                          'src': 'https://sun9-31.userapi.com/c237331/u663156600/d59/-3/s_4327e2112b.jpg',
		                                                          'type': 's',
		                                                          'width': 100},
		                                                         {'height': 160,
		                                                          'src': 'https://sun9-10.userapi.com/c237331/u663156600/d59/-3/o_4327e2112b.jpg',
		                                                          'type': 'o',
		                                                          'width': 220},
		                                                         {'height': 160,
		                                                          'src': 'https://sun9-58.userapi.com/c237331/u663156600/d59/-3/x_4327e2112b.jpg',
		                                                          'type': 'x',
		                                                          'width': 220}]},
		                                     'video': {'file_size': 51487,
		                                               'height': 160,
		                                               'src': 'https://vk.com/doc450410456_528688101?hash=5j7aAssUHNLzDphzUQNtebmyzutvaQoELMMS0vvce8z&dl=GQ2TANBRGA2DKNQ:1713635885:jbSl3DxfEFeFX70Fw8YFPC126aJo2EhMH1HyMGunehk&api=1&mp4=1',
		                                               'width': 220}},
		                         'size': 209532,
		                         'title': '1575491925112.gif',
		                         'type': 3,
		                         'url': 'https://vk.com/doc450410456_528688101?hash=5j7aAssUHNLzDphzUQNtebmyzutvaQoELMMS0vvce8z&dl=DFR6t9LGWd1xJ3UYkUw40iBviVczpF1f1vQlJnx6csH&api=1&no_preview=1'},
		                        {'can_manage': True,
		                         'date': 1540066172,
		                         'ext': 'gif',
		                         'id': 479720483,
		                         'is_unsafe': 0,
		                         'owner_id': 450410456,
		                         'preview': {'photo': {'sizes': [{'height': 74,
		                                                          'src': 'https://sun9-15.userapi.com/c909518/u579603445/d55/-3/m_faf19c3d3c.jpg',
		                                                          'type': 'm',
		                                                          'width': 130},
		                                                         {'height': 57,
		                                                          'src': 'https://sun9-61.userapi.com/c909518/u579603445/d55/-3/s_faf19c3d3c.jpg',
		                                                          'type': 's',
		                                                          'width': 100},
		                                                         {'height': 304,
		                                                          'src': 'https://sun9-75.userapi.com/c909518/u579603445/d55/-3/x_faf19c3d3c.jpg',
		                                                          'type': 'x',
		                                                          'width': 540},
		                                                         {'height': 304,
		                                                          'src': 'https://sun9-23.userapi.com/c909518/u579603445/d55/-3/o_faf19c3d3c.jpg',
		                                                          'type': 'o',
		                                                          'width': 540}]},
		                                     'video': {'file_size': 156575,
		                                               'height': 304,
		                                               'src': 'https://vk.com/doc450410456_479720483?hash=zKZSaRbA2EKSeZcYx3pMQZD7uFCg8NgVQvtwQJFdhZ0&dl=GQ2TANBRGA2DKNQ:1713635885:25iXGIq6gQLTnxvLkC98TCrFX1AQUOZJWp737D2n6nw&api=1&mp4=1',
		                                               'width': 540}},
		                         'size': 2054695,
		                         'title': '1443010288_1883634010.gif',
		                         'type': 3,
		                         'url': 'https://vk.com/doc450410456_479720483?hash=zKZSaRbA2EKSeZcYx3pMQZD7uFCg8NgVQvtwQJFdhZ0&dl=kUKiLTvNUTDWGIlD0JBeektE6b7oZB0uFExRnmUzIW8&api=1&no_preview=1'}]}}
)

print("It did not crash!")

It did not crash!


Можно также соединить все элементы этой схемы воедино, чтобы это был один документ, а не несколько.
Для этого следующая функция перебирает элементы структуры рекурсивно
и ищет те, которые имеют в содержимом только `$ref`;
эти следует заменить на тот объект, на который они ссылаются.

In [21]:
from copy import deepcopy

def attempt_replacements(subtree: dict) -> int:
    # print("subtree:", subtree)
    replacements_done = 0
    for key,value in subtree.items():
        # print("trying value", value)
        if isinstance(value, dict):
            if list(value.keys()) == ['$ref']:
                repl = GLOBAL_RESOURCES[ value["$ref"] ].contents
                # print("Value is a reference -> replacing with", repl)
                subtree[key] = repl
                replacements_done += 1
            else:
                replacements_done += attempt_replacements(value)
        elif isinstance(value, list):
            # print("trying list", value)
            for idx, item in enumerate(value):
                # print("trying item", item)
                if isinstance(item, dict):
                    if list(item.keys()) == ['$ref']:
                        repl = GLOBAL_RESOURCES[ item["$ref"] ].contents
                        # print("List item is a reference -> replacing with", repl)                        
                        value[idx] = repl
                        replacements_done += 1
                    else:
                        replacements_done += attempt_replacements(item)

    
    return replacements_done

def merge_down(schema: dict) -> dict:
    myschema = deepcopy(schema)
    replacements = 1
    while replacements:
        replacements = attempt_replacements(myschema)
        print("Performed", replacements, "replacements")
    
    return myschema

In [22]:
merge_down(global_response_schema)

Performed 2 replacements
Performed 0 replacements


{'$schema': 'https://json-schema.org/draft/2020-12/schema',
 '$id': 'https://vk.com/api/docs.get/response',
 'type': 'object',
 'oneOf': [{'type': 'object',
   'properties': {'error': {'$schema': 'https://json-schema.org/draft/2020-12/schema',
     '$id': 'https://vk.com/api/generic.error',
     'type': 'object',
     'properties': {'error_code': {'type': 'integer',
       'description': 'Код ошибки'},
      'error_msg': {'type': 'string',
       'description': 'Человекочитаемое сообщение об ошибке'},
      'request_params': {'type': 'array',
       'description': 'Копия параметров, которые были отправлены в запросе',
       'items': {'type': 'object',
        'properties': {'key': {'type': 'string'}, 'value': {'type': 'string'}},
        'additionalProperties': False,
        'required': ['key', 'value']}}},
     'required': ['error_code', 'error_msg'],
     'additionalProperties': True}},
   'required': ['error'],
   'additionalProperties': False},
  {'type': 'object',
   'properties

## Заключение

В этой лабораторной работе мы познакомились с JSON-Schema -- 
форматом для описания и валидации JSON-документов.
Такие описания довольно полезны для описания веб-API
и других протоколов, которые работают с JSON-данными,
а также для кодогенерации в языках программирования,
которые лучше работают со статически-типизированными данными.